# DeBERTa-v3 NER for MTA Service Alerts

This notebook implements a Named Entity Recognition (NER) pipeline using deberta-v3-base to extract ROUTE and 
DIRECTION entities from MTA transit alerts.
It uses existing span annotations from the silver dataset.

In [1]:
# Install required packages if missing
%pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=916d9b0da786665095b03f87a500c63cea45bc19ffff91609cb2afc1fedc63c1
  Stored in directory: /root/.cache/pip/wheels/5f/b8/73/0b2c1a76b701a677653dd79ece07cfabd7457989dbfbdcd8d7
Successfully built seqeval


In [2]:
# Run data preparation script
%run 5_prepare_final_data.py


Loading tokenizer: microsoft/deberta-v3-base


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

Added special tokens: ['[ROUTE]', '[/ROUTE]', '[DIR]', '[/DIR]']
Vocab size: 128,005
Saved tokenizer to final_data/tokenizer
Loading data from Preprocessed/MTA_Data_silver_relations.csv...
Loaded 226,160 records

Preparing NER dataset...
  Processed 50,000 / 226,160 records...
  Processed 100,000 / 226,160 records...
  Processed 150,000 / 226,160 records...
  Processed 200,000 / 226,160 records...
  Created NER dataset with 226,160 samples


...


Saved NER dataset to final_data/ner_dataset
  Train: 158,311
  Validation: 33,924
  Test: 33,925

Preparing RE dataset...
  Processed 50,000 / 226,160 records...
  Processed 100,000 / 226,160 records...
  Processed 150,000 / 226,160 records...
  Processed 200,000 / 226,160 records...
  Created RE dataset with 401,453 samples
  Positive pairs: 293,474
  Negative pairs: 107,979


Saving the dataset (0/1 shards):   0%|          | 0/281017 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/60217 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/60219 [00:00<?, ? examples/s]


Saved RE dataset to final_data/re_dataset
  Train: 281,017
  Validation: 60,217
  Test: 60,219

Saved label mappings to final_data/label_mappings.json

DATA PREPARATION COMPLETE

Output directory: final_data/
  ├── tokenizer/          (DeBERTa tokenizer with special tokens)
  ├── ner_dataset/        (Token-level BIO labels)
  ├── re_dataset/         (Entity markers + relation labels)
  └── label_mappings.json (Label ID mappings)

NER Labels:
  0: O
  1: B-ROUTE
  2: I-ROUTE
  3: B-DIRECTION
  4: I-DIRECTION
  -100: [IGNORED] (special tokens)


In [3]:
import json
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForTokenClassification, get_linear_schedule_with_warmup
from torch.optim import AdamW
from datasets import load_from_disk
from seqeval.metrics import classification_report, f1_score
from tqdm.notebook import tqdm
import os

# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

## 1. Data Loading and Preprocessing

In [4]:
# Load the pre-processed dataset from disk
dataset_path = "final_data/ner_dataset"
dataset = load_from_disk(dataset_path)

print("Loaded dataset from disk:")
print(dataset)

Loaded dataset from disk:
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 158311
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 33924
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 33925
    })
})


In [5]:
# Load Label Map
with open("final_data/label_mappings.json", "r") as f:
    mappings = json.load(f)

labels_to_ids = mappings["ner_label2id"]
# JSON keys are strings, convert back to int for id2label
ids_to_labels = {int(k): v for k, v in mappings["ner_id2label"].items()}

print("Label Map:", labels_to_ids)

Label Map: {'O': 0, 'B-ROUTE': 1, 'I-ROUTE': 2, 'B-DIRECTION': 3, 'I-DIRECTION': 4}


## 2. Dataset Class & Tokenization

In [6]:
# Load Tokenizer from local directory
tokenizer_path = "final_data/tokenizer"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

# Set format for PyTorch
# The dataset from disk already contains tokenized inputs
dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

## 3. Data Splits & Class Weights

In [7]:
# Access splits
train_dataset = dataset["train"]
val_dataset = dataset["validation"]
test_dataset = dataset["test"]

print(f"Split sizes: Train: {len(train_dataset)}, Val: {len(val_dataset)}, Test: {len(test_dataset)}")

batch_size = 128

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=4, pin_memory=True)

Split sizes: Train: 158311, Val: 33924, Test: 33925


In [8]:
# Compute Class Weights for Imbalance
# We count occurrences of each label in the training set
label_counts = {id: 0 for id in labels_to_ids.values()}

print("Calculating class weights...")
# Iterate over the dataset directly using DataLoader for efficiency
for batch in tqdm(DataLoader(train_dataset, batch_size=256, num_workers=4), desc="Counting labels"):
    labels = batch['labels'].flatten()
    # Filter out -100
    valid_labels = labels[labels != -100]

    # Update counts
    unique, counts = torch.unique(valid_labels, return_counts=True)
    for lbl, count in zip(unique, counts):
        label_counts[lbl.item()] += count.item()

total_counts = sum(label_counts.values())
num_classes = len(labels_to_ids)

# Enhanced inverse frequency weights with boosting for rare entity types
# Apply extra weight to I-tags and B-DIRECTION which are typically underrepresented
class_weights = []
boost_factors = {
    0: 1.0,   # O - no boost
    1: 1.2,   # B-ROUTE - slight boost
    2: 1.5,   # I-ROUTE - boost for continuity
    3: 1.5,   # B-DIRECTION - boost for rare starts
    4: 1.5    # I-DIRECTION - boost for continuity
}

for i in range(num_classes):
    count = label_counts[i]
    if count > 0:
        # Base inverse frequency weight
        weight = total_counts / (num_classes * count)
        # Apply boost factor
        weight *= boost_factors.get(i, 1.0)
    else:
        weight = 1.0
    class_weights.append(weight)

class_weights = torch.tensor(class_weights, dtype=torch.float)
print("\nLabel distribution in training set:")
for label_name, label_id in labels_to_ids.items():
    count = label_counts[label_id]
    pct = (count / total_counts * 100) if total_counts > 0 else 0
    print(f"  {label_name}: {count:,} ({pct:.2f}%) - weight: {class_weights[label_id]:.3f}")
print(f"\nClass Weights: {class_weights}")

Calculating class weights...


Counting labels:   0%|          | 0/619 [00:00<?, ?it/s]


Label distribution in training set:
  O: 3,730,331 (84.52%) - weight: 0.237
  B-ROUTE: 257,219 (5.83%) - weight: 4.118
  I-ROUTE: 100,533 (2.28%) - weight: 13.170
  B-DIRECTION: 143,432 (3.25%) - weight: 9.231
  I-DIRECTION: 181,980 (4.12%) - weight: 7.276

Class Weights: tensor([ 0.2366,  4.1180, 13.1703,  9.2312,  7.2758])


## 4. Model Configuration & Training Setup

In [9]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('mps') if torch.backends.mps.is_available() else torch.device('cpu')
print(f"Using device: {device}")

# Load Model
model = AutoModelForTokenClassification.from_pretrained(
    'microsoft/deberta-v3-base',
    num_labels=len(labels_to_ids)
)
model.to(device)

# Optimizer with different learning rates
optimizer_grouped_parameters = [
    {'params': model.deberta.parameters(), 'lr': 3e-5},
    {'params': model.classifier.parameters(), 'lr': 1e-4}
]
optimizer = AdamW(optimizer_grouped_parameters)

# Scheduler
epochs = 3 # Start with small number for demo, user can increase
total_steps = len(train_loader) * epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=int(0.1 * total_steps),
    num_training_steps=total_steps
)

# Loss Function with Weights
loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights.to(device))

Using device: cuda


pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

Some weights of DebertaV2ForTokenClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## 5. Training Loop

In [10]:
from torch.cuda.amp import GradScaler

def train_epoch(model, data_loader, optimizer, scheduler, device, loss_fct):
    model.train()
    total_loss = 0
    scaler = GradScaler()  # Initialize gradient scaler for FP16

    for batch in tqdm(data_loader, desc="Training"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()

        # Runs the forward pass with autocasting (FP16)
        with torch.amp.autocast(device_type="cuda", dtype=torch.float16):
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            # Reshape logits and labels for loss calculation
            active_logits = logits.view(-1, len(labels_to_ids))
            active_labels = labels.view(-1)

            loss = loss_fct(active_logits, active_labels)

        total_loss += loss.item()

        # Scales loss. Calls backward() on scaled loss to create scaled gradients.
        scaler.scale(loss).backward()

        # Unscales the gradients of optimizer's assigned params in-place
        scaler.unscale_(optimizer)

        # Since the gradients of optimizer's assigned params are unscaled, clips as usual
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Optimizer's step() is skipped if gradients contain NaNs/Infs
        scaler.step(optimizer)

        # Updates the scale for next iteration
        scaler.update()
        scheduler.step()

    return total_loss / len(data_loader)

def eval_epoch(model, data_loader, device, loss_fct):
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in tqdm(data_loader, desc="Evaluating"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            # Use autocast for evaluation too
            with torch.amp.autocast(device_type="cuda", dtype=torch.float16):
                outputs = model(input_ids=input_ids, attention_mask=attention_mask)
                logits = outputs.logits
                loss = loss_fct(logits.view(-1, len(labels_to_ids)), labels.view(-1))

            total_loss += loss.item()

            # Get predictions
            preds = torch.argmax(logits, dim=2)

            # Convert to list and handle ignores (-100)
            for i in range(len(labels)):
                true_labels = labels[i]
                pred_labels = preds[i]

                true_list = []
                pred_list = []

                for j in range(len(true_labels)):
                    if true_labels[j] != -100:
                        true_list.append(ids_to_labels[true_labels[j].item()])
                        pred_list.append(ids_to_labels[pred_labels[j].item()])

                all_labels.append(true_list)
                all_preds.append(pred_list)

    return total_loss / len(data_loader), all_labels, all_preds

## 6. Execution

In [11]:
best_f1 = 0
patience = 3
patience_counter = 0

for epoch in range(epochs):
    print(f"\nEpoch {epoch + 1}/{epochs}")

    train_loss = train_epoch(model, train_loader, optimizer, scheduler, device, loss_fct)
    print(f"Train Loss: {train_loss:.4f}")

    val_loss, val_labels, val_preds = eval_epoch(model, val_loader, device, loss_fct)

    # Compute Metrics
    f1 = f1_score(val_labels, val_preds)
    report = classification_report(val_labels, val_preds)

    print(f"Val Loss: {val_loss:.4f}")
    print(f"Val F1: {f1:.4f}")
    print("Classification Report:")
    print(report)

    # Save Best Model & Early Stopping
    if f1 > best_f1:
        best_f1 = f1
        patience_counter = 0
        # Save model
        model.save_pretrained("models/deberta_ner_best")
        tokenizer.save_pretrained("models/deberta_ner_best")
        print("New best model saved!")
    else:
        patience_counter += 1
        print(f"No improvement. Patience: {patience_counter}/{patience}")

    if patience_counter >= patience:
        print("Early stopping triggered.")
        break


Epoch 1/3


Training:   0%|          | 0/1237 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/371M [00:00<?, ?B/s]

Train Loss: 0.0871


Evaluating:   0%|          | 0/266 [00:00<?, ?it/s]

...

Val Loss: 0.0017
Val F1: 0.9925
Classification Report:
              precision    recall  f1-score   support

   DIRECTION       1.00      1.00      1.00     30800
       ROUTE       0.98      1.00      0.99     54764

   micro avg       0.99      1.00      0.99     85564
   macro avg       0.99      1.00      0.99     85564
weighted avg       0.99      1.00      0.99     85564

New best model saved!

Epoch 3/3


Training:   0%|          | 0/1237 [00:00<?, ?it/s]

Train Loss: 0.0015


Evaluating:   0%|          | 0/266 [00:00<?, ?it/s]

Val Loss: 0.0015
Val F1: 0.9928
Classification Report:
              precision    recall  f1-score   support

   DIRECTION       1.00      1.00      1.00     30800
       ROUTE       0.98      1.00      0.99     54764

   micro avg       0.99      1.00      0.99     85564
   macro avg       0.99      1.00      0.99     85564
weighted avg       0.99      1.00      0.99     85564

New best model saved!


## 7. Inference Test

In [12]:
def predict_ner(text, model, tokenizer, device):
    model.eval()
    inputs = tokenizer(text, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=2)[0]

    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
    entities = []

    for token, label_id in zip(tokens, preds):
        label = ids_to_labels[label_id.item()]
        if label != "O" and token not in ["[CLS]", "[SEP]"]:
            entities.append((token, label))

    return entities

# Test
test_text = "Jamaica-bound J trains are delayed"
print(predict_ner(test_text, model, tokenizer, device))

[('▁Jamaica', 'B-DIRECTION'), ('-', 'I-DIRECTION'), ('bound', 'I-DIRECTION'), ('▁J', 'B-ROUTE')]


## Evaluation

In [13]:
from seqeval.metrics import precision_score, recall_score

best_dir = "models/deberta_ner_best"
eval_tokenizer = AutoTokenizer.from_pretrained(best_dir)
eval_model = AutoModelForTokenClassification.from_pretrained(best_dir)
eval_model.to(device)

# Use the existing test_dataset (loaded from disk)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

def evaluate_ner(model, data_loader, device):
    loss_fct_eval = torch.nn.CrossEntropyLoss(ignore_index=-100)
    all_true = []
    all_pred = []
    total_loss = 0.0
    model.eval()

    with torch.no_grad():
        for batch in tqdm(data_loader, desc="Evaluating test"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            loss = loss_fct_eval(logits.view(-1, len(labels_to_ids)), labels.view(-1))
            total_loss += loss.item()

            preds = torch.argmax(logits, dim=2)

            for i in range(len(labels)):
                true_seq = []
                pred_seq = []
                for j in range(len(labels[i])):
                    if labels[i][j].item() == -100:
                        continue
                    true_seq.append(ids_to_labels[labels[i][j].item()])
                    pred_seq.append(ids_to_labels[preds[i][j].item()])
                all_true.append(true_seq)
                all_pred.append(pred_seq)

    num_batches = max(len(data_loader), 1)
    return {
        "loss": total_loss / num_batches,
        "precision": precision_score(all_true, all_pred),
        "recall": recall_score(all_true, all_pred),
        "f1": f1_score(all_true, all_pred),
        "report": classification_report(all_true, all_pred)
    }


test_metrics = evaluate_ner(eval_model, test_loader, device)
print(f"Test loss: {test_metrics['loss']:.4f}")
print(f"Test precision: {test_metrics['precision']:.4f}")
print(f"Test recall: {test_metrics['recall']:.4f}")
print(f"Test F1: {test_metrics['f1']:.4f}")
print(test_metrics["report"])


The tokenizer you are loading from 'models/deberta_ner_best' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


Evaluating test:   0%|          | 0/266 [00:00<?, ?it/s]

Test loss: 0.0055
Test precision: 0.9847
Test recall: 0.9993
Test F1: 0.9920
              precision    recall  f1-score   support

   DIRECTION       1.00      1.00      1.00     30863
       ROUTE       0.98      1.00      0.99     54921

   micro avg       0.98      1.00      0.99     85784
   macro avg       0.99      1.00      0.99     85784
weighted avg       0.98      1.00      0.99     85784

